In [2]:
import ee
ee.Authenticate()

Enter verification code:  4/1AdQt8qi9CWcrbuZAfEi-5Kn50gh5ZTeoZjCmeI3wBmHyCFEZ9zVjPoRhVsA



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

# Load data

In [5]:
# read Tree Mosaic Land
tml = ee.ImageCollection('projects/wri-datalab/TML')
tml_tree_cover = tml.reduce(ee.Reducer.mean()).rename('b1')

In [6]:
# get list of cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/geo_ref.csv')
boundary_georef.head()

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4


In [7]:
# remove cities without tree cover data availability
tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM2,COD,ADM2
1,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
2,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
3,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
4,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
5,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
6,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
7,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
8,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Extract TML by city

In [9]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name'] + '-' + boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for TML as GeoTIFF
    geemap.ee_export_image_to_drive(
        tml_tree_cover, 
        folder='data',
        # # 10m resolution
        # description = boundary_id + '-TML-TreeCover',
        # scale=10, 
         # 50m resolution
        description = boundary_id + '-TML-TreeCover_50m',
        scale=50, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

0
COD-Bukavu-ADM3union
1
COD-Uvira-ADM3union
2
COG-Brazzaville-ADM4union
3
COL-Barranquilla-ADM4union
4
ETH-Addis_Ababa-ADM4union
5
ETH-Dire_Dawa-ADM3union
6
KEN-Nairobi-ADM3union
7
MDG-Antananarivo-ADM4union
8
MEX-Mexico_City-ADM2union


# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [10]:
# connect to s3
aws_credentials = pd.read_csv('C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\aws\\credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

# specify bucket name
bucket_name = 'cities-cities4forests' 

In [11]:
out_dir = os.getcwd()

In [14]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data\\' + boundary_id +'-TML-TreeCover_50m.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/tree_cover/tree_mosaic_land/v_0/'+ boundary_id + '-TML-tree_cover-2020_50m.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Mexico_City-ADM2union


# Visualize

In [41]:
## create map
Map = geemap.Map(height="350px")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [85]:
i = 10
boundary_id = geo_ref.loc[i, 'geo_name'] + '-' + geo_ref.loc[i, 'aoi_boundary_name']
print(boundary_id)

MEX-Mexico_City-ADM2union


In [86]:
# read boundaries
boundary_path = 'https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-'+boundary_id+'.geojson'
boundary_geo = requests.get(boundary_path).json()
boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)

In [87]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(boundary_geo_ee, zoom=12)

In [88]:
Map.addLayer(boundary_geo_ee, {'color': 'FF0000'}, 'City boundary')

In [44]:
Map.addLayer(tml_tree_cover,
             {'min':0, 'max':0.5, 'palette':['white','#006400']},
             'Tree Cover 2020 (WRI Trees in Mosaic Landscapes)',True,1)